In [17]:
import torch
import torch_directml


# Select DirectML device
device = torch_directml.device()

# Check if DirectML device is available
print("DirectML device available:", device is not None)

# Print device name
if device is not None:
    print("Device name:", "AMD GPU")


DirectML device available: True
Device name: AMD GPU


In [23]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

# Constants
data_url = 'https://raw.githubusercontent.com/KeithGalli/pandas/master/pokemon_data.csv'
BATCH_SIZE = 32
TEST_SIZE = 0.25
RANDOM_STATE = 42
PATIENCE = 5
MAX_EPOCHS = 1000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load dataset
df = pd.read_csv(data_url, index_col='Name')
df = df.drop(['#'], axis=1)

# Map Legendary to binary values
df['Legendary'] = df['Legendary'].map({False: 0, True: 1})

# One-hot encode categorical variables and handle missing values
df = pd.get_dummies(df, columns=['Type 1', 'Type 2'], dummy_na=True)

# Separate features and target
x_columns = df.drop('Legendary', axis=1).columns
x = df[x_columns].astype(float).values
y = df['Legendary'].values

# Convert to PyTorch tensors
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Split into train/test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# Move to device (GPU or CPU)
def move_to_device(*args):
    return [t.to(DEVICE) for t in args]

x_train, y_train, x_test, y_test = move_to_device(x_train, y_train, x_test, y_test)

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [19]:

# Define the neural network model
class PokemonModel(nn.Module):
    def __init__(self, input_dim):
        super(PokemonModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

model = PokemonModel(x_train.shape[1]).to(DEVICE)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training function with early stopping
def train_model(model, train_loader, test_loader, criterion, optimizer, max_epochs, patience):
    best_loss = float('inf')
    no_improvement = 0

    for epoch in range(max_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = sum(criterion(model(inputs), labels).item() for inputs, labels in test_loader) / len(test_loader)
        
        if val_loss < best_loss - 1e-3:
            best_loss = val_loss
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print(f"Early stopping at epoch {epoch}")
            break
    
    return model

model = train_model(model, train_loader, test_loader, criterion, optimizer, MAX_EPOCHS, PATIENCE)

# Evaluate model
def evaluate_model(model, x_test, y_test):
    model.eval()
    with torch.no_grad():
        pred = model(x_test).cpu().numpy().flatten()
        pred = np.clip(pred, a_min=1e-6, a_max=1-1e-6)
        logloss = metrics.log_loss(y_test.cpu(), pred)
        pred_binary = (pred > 0.5).astype(int)
        accuracy = metrics.accuracy_score(y_test.cpu().numpy(), pred_binary)
    return logloss, accuracy

logloss, accuracy = evaluate_model(model, x_test, y_test)
print(f"Validation logloss: {logloss}")
print(f"Validation accuracy score: {accuracy}")



Early stopping at epoch 27
Validation logloss: 0.28385608108165034
Validation accuracy score: 0.89


In [24]:
# Define perturbation rank function
def perturbation_rank(device, model, x_test, y_test, feature_names, verbose=False):
    model.eval()
    baseline_loss = torch.nn.BCELoss()(model(x_test), y_test).item()
    importance_scores = []

    for i in range(x_test.shape[1]):
        x_test_perturbed = x_test.clone()
        # Shuffle the tensor column
        x_test_perturbed[:, i] = x_test_perturbed[torch.randperm(x_test_perturbed.size(0)), i]
        perturbed_loss = torch.nn.BCELoss()(model(x_test_perturbed), y_test).item()
        importance = perturbed_loss - baseline_loss
        importance_scores.append(importance)

        if verbose:
            print(f"Feature {feature_names[i]} - Perturbed Loss: {perturbed_loss:.4f} - Importance: {importance:.4f}")

    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance_scores
    }).sort_values(by='Importance', ascending=False).reset_index(drop=True)

    return importance_df

# Display feature importance
def get_feature_importance(df_train, model, x_test, y_test, feature_names):
    rank = perturbation_rank(DEVICE, model, x_test, y_test, feature_names, False)
    display(rank[0:10])

feature_names = df.drop('Legendary', axis=1).columns.tolist()
get_feature_importance(df, model, x_test, y_test, feature_names)

,Feature,Importance
0,Attack,0.155011
1,Sp. Atk,0.136268
2,Sp. Def,0.023644
3,Type 2_Poison,0.012382
4,Defense,0.007841
5,Type 1_Poison,0.007367
6,Type 1_Grass,0.006897
7,Type 1_Water,0.006422
8,Type 1_Fire,0.003202
9,Type 1_Dark,0.002299
